# Homework 2

## 1. Merge sorted stream

Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.
Bonus point: can you make it generic such that it can merge any number of streams?       

I provide two different methods for this question. Both of them directly consider the general case: 
with any number of streams

### Method 1

In [23]:
# We transform streams to list and use list sorting
# Here we apply variadic parameter

def merge_sorted_stream(*streams):
    # transfer every stream to list form, for the convenience of sorting
    lists = [list(stream) for stream in streams]
    # merge these lists by list comprehension
    merged_list = [k[i] for k in lists for i in range(len(k))]
    # sorted the merged_list
    sorted_list = sorted(merged_list)
    
    for element in sorted_list:
        yield element

In [24]:
# Test case 1
stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0
1
2
3
4
5
6
7
8
9


In [25]:
# Test case 2
def gen_135():
    yield 1
    yield 3
    yield 5

def gen_404():
    yield (-4)
    yield 0
    yield 4

gen1 = gen_135()
gen2 = gen_404()
gen3 = iter(range(2,10,2))
for x in merge_sorted_stream(gen1, gen2, gen3):
    print(x)

-4
0
1
2
3
4
4
5
6
8


### Method 2

In [26]:
# Here we use the property of generators: 'next' function
def merge_sorted_stream(*streams):
    
    str_len = len(streams)
    
    # cur_value means the number generated by each generator
    cur_value = [0]*str_len
    
    # terminated denotes whether this generator has been used up
    terminated = [0]*str_len
    
    # obtain the initial value for each generator
    for i in range(str_len):
        cur_value[i] = next(streams[i])
    
    # when not all of the generators has been terminated
    while 0 in terminated:
        
        # look for the minimum number and its position
        cur_min = 99999
        cur_pos = 0
        for i in range(0, str_len):
            if terminated[i] == 0 and cur_value[i] < cur_min:
                cur_pos = i
                cur_min = cur_value[i]
        
        # generate this minimum number
        yield cur_min
        
        # update the number if we can
        try:
            cur_value[cur_pos] = next(streams[cur_pos])
        except StopIteration:
            terminated[cur_pos] = 1
        

In [27]:
# Test case 1
stream1 = iter(range(0, 10, 2))
stream2 = iter(range(1, 10, 2))

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0
1
2
3
4
5
6
7
8
9


In [28]:
# Test case 2
def gen_135():
    yield 1
    yield 3
    yield 5

def gen_404():
    yield (-4)
    yield 0
    yield 4

gen1 = gen_135()
gen2 = gen_404()
gen3 = iter(range(2,10,2))
for x in merge_sorted_stream(gen1, gen2, gen3):
    print(x)

-4
0
1
2
3
4
4
5
6
8


## Tree traversal

Define a Tree class with a method that can walk through the tree in different orders. 
Hint: use generator will make your life a lot easier.

In [33]:
class TreeNode:
    
    # Constructor
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def in_order(self):
        if self.left:
            yield from (self.left).in_order()
        yield str(self.value)
        if self.right:
            yield from (self.right).in_order()

    def pre_order(self):
        yield str(self.value)
        if self.left:
            yield from (self.left).pre_order()
        if self.right:
            yield from (self.right).pre_order()

    def post_order(self):
        if self.left:
            yield from (self.left).post_order()
        if self.right:
            yield from (self.right).post_order()
        yield str(self.value)

In [34]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

print(' -> '.join(item for item in root.in_order()))

4 -> 2 -> 5 -> 1 -> 3


## Implement a timer

Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class?

In [37]:
import time

class timer:
    
    # work as a decorator
    def __call__(self, func):
        
        def inner(x):
            self.start_time = time.time()
            func(x)
            self.end_time = time.time()
            print(f"Total execution time --- {self.end_time - self.start_time} seconds ---")
            
        return inner
    
    # 'with' method
    def __enter__(self):
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_value ,traceback):
        self.end_time = time.time()
        print(f"Total execution time --- {self.end_time - self.start_time} seconds ---")
        return True

In [38]:
with timer() as timer:
    time.sleep(3)
    
@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(3)

Total execution time --- 3.002317190170288 seconds ---
Total execution time --- 3.0050768852233887 seconds ---
